# Preprocessing

In [33]:
#from helpers import *
import pandas as pd
import numpy as np
import re
from preprocessing import *
from textblob import TextBlob

import os

In [2]:
re

<module 're' from '/Users/evaluvison/opt/miniconda3/lib/python3.7/re.py'>

In [3]:
dfs = []
for r, d, f in os.walk(os.getcwd()):
    for file in f:
        if 'withheldtweets.json' in file:
            dfs.append(pd.read_json("./censored_tweets/%s" % file, lines=True))

df_cen = pd.concat(dfs)
df_cen = df_cen.dropna(subset=['withheld_in_countries'])
df_cen.head(10)

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,filter_level,lang,timestamp_ms,linked,display_text_range,withheld_in_countries,extended_entities,possibly_sensitive,retweeted_status,withheld_copyright
1,2021-02-26 10:48:35+00:00,1365252415444946945,1365252415444946944,#Balakot \nPak Army is our pride ❤️👍 https://t...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,en,2021-02-26 10:48:35.662,no,"[0, 35]",[IN],"{'media': [{'id': 1365252409015033857, 'id_str...",0.0,NaN,NaN
2,2021-02-26 10:45:51+00:00,1365251727574900738,1365251727574900736,RT @ZaidZamanHamid: لوگوں کی اکثریت ہدایت نہیں...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,ur,2021-02-26 10:45:51.661,no,NaN,[IN],NaN,NaN,{'created_at': 'Thu Feb 25 18:59:12 +0000 2021...,NaN
3,2021-02-26 10:45:51+00:00,1365251727574900738,1365251727574900736,RT @ZaidZamanHamid: لوگوں کی اکثریت ہدایت نہیں...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,ur,2021-02-26 10:45:51.661,retweeted,NaN,[IN],NaN,NaN,{'created_at': 'Thu Feb 25 18:59:12 +0000 2021...,NaN
4,2021-02-26 10:38:57+00:00,1365249991137251328,1365249991137251328,RT @SaniaNishtar: سید ابراہیم کا تعلق قبائلی ض...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,ur,2021-02-26 10:38:57.662,no,NaN,[IN],NaN,NaN,{'created_at': 'Fri Feb 26 05:23:00 +0000 2021...,NaN
5,2021-02-26 10:10:45+00:00,1365242894362279938,1365242894362279936,RT @mosa_abumarzook: في مثل فجر هذا اليوم قبل ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,...,low,ar,2021-02-26 10:10:45.659,no,NaN,[IL],NaN,NaN,{'created_at': 'Thu Feb 25 19:04:40 +0000 2021...,NaN
6,2021-02-26 10:10:45+00:00,1365242894362279938,1365242894362279936,RT @mosa_abumarzook: في مثل فجر هذا اليوم قبل ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,...,low,ar,2021-02-26 10:10:45.659,retweeted,NaN,[IL],NaN,NaN,{'created_at': 'Thu Feb 25 19:04:40 +0000 2021...,NaN
7,2021-02-26 10:10:46+00:00,1365242898569134080,1365242898569134080,RT @Saimhun: Mujhe aj bhi batting ki bari ni d...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,hi,2021-02-26 10:10:46.662,no,NaN,[IN],"{'media': [{'id': 1365238103892516864, 'id_str...",0.0,{'created_at': 'Fri Feb 26 09:51:44 +0000 2021...,NaN
8,2021-02-26 10:10:46+00:00,1365242898569134080,1365242898569134080,RT @Saimhun: Mujhe aj bhi batting ki bari ni d...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,hi,2021-02-26 10:10:46.662,retweeted,NaN,[IN],"{'media': [{'id': 1365238103892516864, 'id_str...",0.0,{'created_at': 'Fri Feb 26 09:51:44 +0000 2021...,NaN
9,2021-02-26 10:59:24+00:00,1365255137552457730,1365255137552457728,RT @iVeenaKhan: (اے پیغمبرﷺ! لوگوں سے) کہہ دو ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,ur,2021-02-26 10:59:24.663,no,NaN,[IN],NaN,NaN,{'created_at': 'Fri Feb 26 05:42:25 +0000 2021...,NaN
10,2021-02-26 10:59:24+00:00,1365255137552457730,1365255137552457728,RT @iVeenaKhan: (اے پیغمبرﷺ! لوگوں سے) کہہ دو ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,low,ur,2021-02-26 10:59:24.663,retweeted,NaN,[IN],NaN,NaN,{'created_at': 'Fri Feb 26 05:42:25 +0000 2021...,NaN


In [6]:
pd.options.display.max_colwidth = 500
pd.options.display.width = 300
df_english = df_cen[df_cen['lang'] == 'en']

In [11]:
list_of_languages = df_cen['lang'].unique()
list_of_languages

array(['en', 'ur', 'ar', 'hi', 'und', 'cs', 'tr', 'in', 'de', 'da', 'es',
       'ht', 'fr', 'tl', 'hu', 'uk', 'sd', 'bn', 'my', 'nl', 'et', 'ro',
       'pa', 'fa', 'th', 'ja', 'fi', 'el', 'pt', 'it', 'zh', 'vi', 'ckb',
       'ca', 'pl', 'sv', 'lt', 'cy', 'lv', 'km', 'ko', 'mr', 'ru', 'lo',
       'ps', 'sl', 'ta', 'no', 'is', 'eu'], dtype=object)

In [60]:
def create_csv_traduction(dataframe):
    list_en=[]
    texts = list(dataframe['text'])
    langs = list(dataframe['lang'])
    for s in dataframe.shape[0]:
        if langs[s] != 'und':
            result = translator.translate(texts[s], "english", langs[s])
            list_en.append(result)
    
    #df_new = dataframe[dataframe['lang'] != 'und']
    #df_new['Translation'] = list_en
    return pd.DataFrame(list_en, columns = 'translation')
    #df_new.to_csv(".\EnglishTranslation.csv")

In [61]:
df_test = df_cen.head(100)
df_gogogo = create_csv_traduction(df_test)
print('done')

TypeError: 'int' object is not iterable

In [58]:
def lower_text(df_text):
    return df_text.apply(lambda x: x.lower())

In [54]:
df_test = df_cen.head(10)

In [59]:
lower_test = lower_text(df_cen['text'])
lower_test

['#balakot \npak army is our pride ❤️👍 https://t.co/jirgpsmmgo',
 'rt @zaidzamanhamid: لوگوں کی اکثریت ہدایت نہیں لینا چاہ رہی۔ \nہمارے فیس بک پیج پر 13 لاکھ ممبر ہیں۔ ہمارے ٹویٹر پیج پر سوا تین لاکھ۔۔\nاس کے…',
 'rt @zaidzamanhamid: لوگوں کی اکثریت ہدایت نہیں لینا چاہ رہی۔ \nہمارے فیس بک پیج پر 13 لاکھ ممبر ہیں۔ ہمارے ٹویٹر پیج پر سوا تین لاکھ۔۔\nاس کے…',
 'rt @sanianishtar: سید ابراہیم کا تعلق قبائلی ضلع کرم سے ہے۔ احساس بلا سود قرضہ  لے کر انہوں  نے اپنا سبزی کا کاروبار بڑھایا ہے جس سے ان کی…',
 'rt @mosa_abumarzook: في مثل فجر هذا اليوم قبل 27 عامًا، قدّم المسجد الإبراهيمي ثلةً من الشهداء، وهم ركّع سجود. \nستبقى #ذكرى_مجزرة_الحرم_الإ…',
 'rt @mosa_abumarzook: في مثل فجر هذا اليوم قبل 27 عامًا، قدّم المسجد الإبراهيمي ثلةً من الشهداء، وهم ركّع سجود. \nستبقى #ذكرى_مجزرة_الحرم_الإ…',
 'rt @saimhun: mujhe aj bhi batting ki bari ni di😔\n yahin so jata hun! subha jb sab ayenge pehle pehle me kheilunga🤔🤔😁 https://t.co/0pjhxburfb',
 'rt @saimhun: mujhe aj bhi batting ki bari ni di😔\n yahin 

In [62]:
df_cen[df_cen['lang']=='und']

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,filter_level,lang,timestamp_ms,linked,display_text_range,withheld_in_countries,extended_entities,possibly_sensitive,retweeted_status,withheld_copyright
12,2021-02-26 10:20:07+00:00,1365245251581976580,1365245251581976576,RT @HTAustralia: #أقيموا_الخلافة\n#ReturnTheKhilafah\n#YenidenHilafet\n#خلافت_کو_قائم_کرو\n#TurudisheniKhilafah\n#EtablerKhilafah\n#УстановитеХал…,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",False,NaN,NaN,NaN,NaN,...,low,und,2021-02-26 10:20:07.664,no,NaN,[RU],NaN,NaN,"{'created_at': 'Fri Feb 26 09:43:58 +0000 2021', 'id': 1365236154119970818, 'id_str': '1365236154119970818', 'text': '#أقيموا_الخلافة #ReturnTheKhilafah #YenidenHilafet #خلافت_کو_قائم_کرو #TurudisheniKhilafah #EtablerKhilafah… https://t.co/SmtZWCm8Ui', 'display_text_range': [0, 140], 'source': '<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>', 'truncated': True, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None,...",NaN
13,2021-02-26 10:20:07+00:00,1365245251581976580,1365245251581976576,RT @HTAustralia: #أقيموا_الخلافة\n#ReturnTheKhilafah\n#YenidenHilafet\n#خلافت_کو_قائم_کرو\n#TurudisheniKhilafah\n#EtablerKhilafah\n#УстановитеХал…,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",False,NaN,NaN,NaN,NaN,...,low,und,2021-02-26 10:20:07.664,retweeted,NaN,[RU],NaN,NaN,"{'created_at': 'Fri Feb 26 09:43:58 +0000 2021', 'id': 1365236154119970818, 'id_str': '1365236154119970818', 'text': '#أقيموا_الخلافة #ReturnTheKhilafah #YenidenHilafet #خلافت_کو_قائم_کرو #TurudisheniKhilafah #EtablerKhilafah… https://t.co/SmtZWCm8Ui', 'display_text_range': [0, 140], 'source': '<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>', 'truncated': True, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None,...",NaN
16,2021-02-26 10:51:37+00:00,1365253178812338176,1365253178812338176,RT @hizbuttahrir: #ReturnTheKhilafah #TimeForIslam #TimeForKhilafah #HTBritain https://t.co/iSG0PvcopD,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",False,NaN,NaN,NaN,NaN,...,low,und,2021-02-26 10:51:37.663,no,NaN,[RU],"{'media': [{'id': 1364893064586747905, 'id_str': '1364893064586747905', 'indices': [79, 102], 'media_url': 'http://pbs.twimg.com/media/EvETAr8WYAEpbOx.jpg', 'media_url_https': 'https://pbs.twimg.com/media/EvETAr8WYAEpbOx.jpg', 'url': 'https://t.co/iSG0PvcopD', 'display_url': 'pic.twitter.com/iSG0PvcopD', 'expanded_url': 'https://twitter.com/hizbuttahrir/status/1364893067464101889/photo/1', 'type': 'photo', 'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'small': {'w': 680, 'h': 68...",0.0,"{'created_at': 'Thu Feb 25 11:00:40 +0000 2021', 'id': 1364893067464101889, 'id_str': '1364893067464101889', 'text': '#ReturnTheKhilafah #TimeForIslam #TimeForKhilafah #HTBritain https://t.co/iSG0PvcopD', 'display_text_range': [0, 60], 'source': '<a href=""https://buffer.com"" rel=""nofollow"">Buffer</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'i...",NaN
17,2021-02-26 10:51:37+00:00,1365253178812338176,1365253178812338176,RT @hizbuttahrir: #ReturnTheKhilafah #TimeForIslam #TimeForKhilafah #HTBritain https://t.co/iSG0PvcopD,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",False,NaN,NaN,NaN,NaN,...,low,und,2021-02-26 10:51:37.663,retweeted,NaN,[RU],"{'media': [{'id': 1364893064586747905, 'id_str': '1364893064586747905', 'indices': [79, 102], 'media_url': 'http://pbs.twimg.com/media/EvETAr8WYAEpbOx.jpg', 'media_url_https': 'https://pbs.twimg.com/media/EvETAr8WYAEpbOx.

In [ ]:
def remove_punctuation(words):
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', (word))
        if new_word != '':
            new_words.append(new_word)
    return new_words

In [47]:
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/evaluvison/nltk_data...
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/evaluvison/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/evaluvison/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
def spelling_correction(tweets):
    textBlb = TextBlob(tweets)           
    return textBlb.correct()

In [50]:
def remove_char(tweets):
    return re.sub(r"[^a-zA-Z0-9# ]", " ", tweets)

stop_words = set(stopwords.words('english'))
lemmatizer = nltk.stem.WordNetLemmatizer()
w_tokenizer = TweetTokenizer()

def lemmatize_text(text):
    return [(lemmatizer.lemmatize(w)) for w in w_tokenizer.tokenize((text))]
    

def clean_tweets_after_trad(df_text):
    clean_tweets = df_text.apply(lambda x: remove_char(x))
    clean_tweets = pd.DataFrame([ele for ele in clean_tweets if ele != ''], columns =['text'])['text']
    clean_tweets = clean_tweets.apply(lambda x: x.lower())
    #clean_tweets = clean_tweets.apply(lambda x: spelling_correction(x))
    clean_tweets = clean_tweets.apply(lambda x : lemmatize_text(x))
    clean_tweets = clean_tweets.apply(lambda x: [item for item in x if item not in stop_words])

In [67]:
lemmatizer = nltk.stem.WordNetLemmatizer()
w_tokenizer = TweetTokenizer()
def lemmatize_text(text):
    return [(lemmatizer.lemmatize(w)) for w in w_tokenizer.tokenize((text))]

### Pour Robin ###

In [51]:
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/evaluvison/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/evaluvison/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/evaluvison/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [52]:
def remove_char(tweets):
    return re.sub(r"[^a-zA-Z0-9# ]", " ", tweets)

stop_words = set(stopwords.words('english'))
lemmatizer = nltk.stem.WordNetLemmatizer()
w_tokenizer = TweetTokenizer()

def lemmatize_text(text):
    return [(lemmatizer.lemmatize(w)) for w in w_tokenizer.tokenize((text))]
    

def clean_tweets_after_trad(df_text):
    clean_tweets = df_text.apply(lambda x: remove_char(x))
    clean_tweets = pd.DataFrame([ele for ele in clean_tweets if ele != ''], columns =['text'])['text']
    clean_tweets = clean_tweets.apply(lambda x: x.lower())
    clean_tweets = clean_tweets.apply(lambda x : lemmatize_text(x))
    clean_tweets = clean_tweets.apply(lambda x: [item for item in x if item not in stop_words])

### Fin ###

In [12]:
import re

In [7]:
df_txt_clean = clean_tweets(df_en['text'])
df_txt_clean 

1                         \nPak Army is our pride ❤️👍  
11      Ben Dunk "If you haven't seen Lahore, you ha...
15       \n\nIndia has also bestowed Abhinandan Vart...
28      After the great Companion Saad bin Muadh emb...
29      After the great Companion Saad bin Muadh emb...
30      Paying homage to one of the great pilots in ...
31      Paying homage to one of the great pilots in ...
33           STRIP CHAT ⭐️  slTo\nSTRIP CHAT ⭐️  slTo  
34      Champions of Democracy are in fact champions...
35       \nMilitant commander Nooristan known as Has...
Name: text, dtype: object

In [18]:
pd.DataFrame(df_txt_clean)

,text
1,\nPak Army is our pride ❤️👍
11,"Ben Dunk ""If you haven't seen Lahore, you ha..."
15,\n\nIndia has also bestowed Abhinandan Vart...
28,After the great Companion Saad bin Muadh emb...
29,After the great Companion Saad bin Muadh emb...
30,Paying homage to one of the great pilots in ...
31,Paying homage to one of the great pilots in ...
33,STRIP CHAT ⭐️ slTo\nSTRIP CHAT ⭐️ slTo
34,Champions of Democracy are in fact champions...
35,\nMilitant commander Nooristan known as Has...


In [49]:
df_clean_after = clean_tweets_after_trad(pd.DataFrame(df_txt_clean)['text'])
df_clean_after

0                                   [pak, army, pride]
1    [ben, dunk, havent, seen, lahore, havent, even...
2    [india, ha, also, bestowed, abhinandan, vartha...
3    [great, companion, saad, bin, muadh, embraced,...
4    [great, companion, saad, bin, muadh, embraced,...
5    [paying, homage, one, great, pilot, historypri...
6    [paying, homage, one, great, pilot, historypri...
7                 [strip, chat, sltostrip, chat, slto]
8    [champion, democracy, fact, champion, fascism,...
9    [militant, commander, nooristan, known, hasan,...
Name: text, dtype: object


# BTM

In [1]:
import bitermplus as btm
import numpy as np
import pandas as pd

In [2]:
#my_df = pd.read_csv('./out_clean.csv')
my_df = pd.read_csv('data/to_be_clustered.csv.gz', compression="gzip")

In [6]:
my_df['clean'].apply(lambda x: str(x))
in_df = my_df[my_df['whcs'] == "France"]
in_df = in_df['clean'].dropna()
texts = in_df.str.strip().tolist()

In [7]:
# PREPROCESSING
# Obtaining terms frequency in a sparse matrix and corpus vocabulary
X, vocabulary, vocab_dict = btm.get_words_freqs(texts)
tf = np.array(X.sum(axis=0)).ravel()
# Vectorizing documents
docs_vec = btm.get_vectorized_docs(texts, vocabulary)
docs_lens = list(map(len, docs_vec))
# Generating biterms
biterms = btm.get_biterms(docs_vec)

In [125]:
# INITIALIZING AND RUNNING MODEL
#model = btm.BTM(
#    X, vocabulary, seed=12321, T=8, M=20, alpha=50/8, beta=0.01)
# 1000, 8, 20, 10/8, 0,1
model = btm.BTM(
    X, vocabulary, seed=1000, T=8, M=20, alpha=0.1, beta=0.5)
model.fit(biterms, iterations=20)
p_zd = model.transform(docs_vec)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 900/900 [00:00<00:00, 99283.91it/s]


In [126]:
# METRICS
perplexity = btm.perplexity(model.matrix_topics_words_, p_zd, X, 8)
coherence = btm.coherence(model.matrix_topics_words_, X, M=20)

In [127]:
perplexity

608.6595799632812

In [128]:
coherence

array([-518.37615514, -514.06894634, -425.21398672, -401.14758631,
       -420.43377282, -427.23992696, -396.88426776, -461.08165752])

In [129]:
p_zd

array([[1.48600495e-01, 6.00354387e-02, 3.31941227e-02, ...,
        1.31019365e-02, 5.73658098e-01, 1.59209861e-01],
       [7.84140604e-01, 5.43217040e-03, 8.00191570e-02, ...,
        2.48193490e-02, 5.63695044e-02, 3.35423855e-02],
       [9.22518396e-01, 3.92789817e-03, 3.15409762e-04, ...,
        1.53426996e-03, 3.68656224e-03, 4.55190452e-02],
       ...,
       [7.57072023e-01, 1.06639289e-02, 1.07136169e-02, ...,
        8.75272965e-03, 6.19906909e-02, 7.73397004e-02],
       [6.41332740e-01, 1.03512394e-02, 4.20997597e-03, ...,
        1.37769292e-03, 4.49340958e-02, 2.16277759e-02],
       [3.71972462e-02, 6.18836395e-03, 8.86660752e-01, ...,
        1.01104718e-02, 2.69359860e-03, 1.98251216e-02]])

In [130]:
top_words = btm.get_top_topic_words(model, words_num=20)
top_words

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7
0,white,one,woman,year,people,like,muslim,biden
1,people,woman,amp,new,left,great,attack,trump
2,get,right,sharia,twitter,minister,soral,nigga,amp
3,black,black,one,two,someone,expose,lil,left
4,say,today,country,follower,play,left,islamic,warn
5,anti,people,state,another,prime,reset,terrorist,germany
6,go,time,let,face,continue,good,amp,wing
7,know,never,omar,show,political,farmer,police,break
8,like,even,citizen,migrant,migration,alain,france,fight
9,many,want,law,student,border,power,migrant,america


### Topics 
We can give categories we used for labelling to these clusters
1. 'white/black - racism', 'women' 0
1. 'jew - antisemtism' 2
1. 'tweeter acccounts', 'communism - against bankers , against state  prder ' 3
1. 'immigration' 4
1. 'farright', 'Alain Soral', 'communism - against bankers , against state  prder ' 5
1. 'islam', 'terrorism' 6
1. 'american elections' 7





1. 'homophobia'

1. Other

Don't have 
- 'India - Pakistan'
- 'coronavirus - vaccination'
- 'fantasy sex play'
- 'homophobia'

In [30]:
# RESULTS VISUALIZATION
# You need to install tmplot first
import tmplot as tmp
tmp.report(model=model, docs=texts,  show_headers = True,show_docs= False,show_topics= False, height=500, width= 500)

In [17]:
test=model.labels_
test

array([0, 2, 2, 5, 6, 2, 7, 8, 7, 0, 4, 4, 4, 2, 5, 2, 1, 2, 0, 2, 4, 5,
       5, 3, 6, 6, 6, 2, 9, 9, 3, 2, 5, 2, 3, 3, 0, 5, 0, 2, 1, 8, 0, 5,
       1, 0, 2, 9, 6, 6, 6, 0, 8, 9, 2, 6, 4, 9, 8, 2, 9, 4, 3, 2, 3, 1,
       4, 0, 0, 0, 6, 5, 4, 2, 4, 5, 9, 2, 2, 7, 2, 9, 9, 8, 3, 3, 8, 2,
       5, 1, 1, 7, 2, 3, 1, 1, 6, 9, 4, 2, 1, 1, 2, 9, 5, 2, 9, 2, 2, 1,
       0, 7, 9, 2, 8, 3, 1, 0, 2, 7, 3, 7, 5, 6, 5, 6, 5, 5, 3, 2, 2, 8,
       2, 0, 1, 2, 1, 0, 2, 0, 5, 9, 3, 7, 1, 4, 3, 3, 4, 2, 8, 7, 5, 5,
       5, 2, 2, 8, 1, 1, 0, 2, 2, 3, 1, 7, 5, 3, 3, 1, 5, 5, 7, 2, 6, 5,
       0, 0, 8, 7, 8, 0, 2, 2, 3, 2, 3, 1, 2, 4, 6, 3, 6, 1, 5, 2, 6, 0,
       2, 1, 9, 7, 9, 1, 1, 2, 2, 6, 7, 4, 8, 4, 4, 8, 4, 2, 3, 8, 1, 2,
       7, 3, 0, 3, 9, 2, 1, 3, 1, 6, 8, 2, 0, 5, 4, 7, 4, 2, 6, 5, 5, 3,
       1, 1, 3, 1, 8, 2, 7, 1, 1, 7, 2, 8, 2, 0, 8, 1, 1, 5, 7, 8, 8, 4,
       1, 9, 7, 4, 7, 7, 2, 3, 3, 4, 2, 6, 8, 3, 0, 2, 2, 2, 8, 4, 1, 1,
       9, 5, 0, 8, 2, 1, 1, 0, 6, 4, 5, 8, 1, 0, 2,

In [16]:
model.matrix_docs_topics_[0]

array([6.86150859e-01, 2.93460920e-02, 6.10523066e-02, 1.34538339e-02,
       3.22713298e-02, 2.61853550e-04, 1.73056371e-03, 2.08317691e-02,
       1.28493416e-01, 2.64079761e-02])

In [131]:
from gensim.models.coherencemodel import CoherenceModel


In [133]:
cm = CoherenceModel(model=model, coherence='u_mass')
coherence = cm.get_coherence()  # get coherence value

ValueError: One of texts or corpus has to be provided.